### This program corresponds to the fifth experiment of the article, which solves the Laplace equation problem in different triangular domains

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable as v
import scipy.special as scp
import time

is_gpu = torch.cuda.is_available()
if is_gpu:
    id = 0
    torch.cuda.set_device(id)
    
#gpu_nums = torch.cuda.device_count()
#gpu_index = torch.cuda.current_device()
#print(is_gpu,gpu_nums,gpu_index)
device = torch.device('cuda' if is_gpu else 'cpu')

device = torch.device('cpu')
print(device)
torch.set_default_tensor_type('torch.DoubleTensor')
print(torch.__version__)

import sys
sys.path.append('../Models')
sys.path.append('../Utils')

import Network as Nw
import BoundaryCondition as Bdc
import Loss_functions as lf
import DomainSetting as Ds
import GenerateIntegralMatrix as IM
import Training as Tn



cpu
1.6.0+cu101


In [ ]:
# Set problem domain and boundary condition
# Generate integral points on boundary and the integral matrix of the fundamental solution 

# M           Number of samples taken on each edge
# sample_tri  Number of samples of triangle
# vertex      Vertex coordinates of each triangle
M = 60
sample_tri = 80
line = (torch.linspace(0,1-1/M,M)).reshape(-1,1)

vertex_num = 3
vertex_sample = torch.rand(sample_tri,3)
vertex = torch.zeros(sample_tri,vertex_num,2)
for i in range(sample_tri):
    vertex[i,1,0] = vertex_sample[i,0]
    vertex[i,2,:] = vertex_sample[i,1:3]


In [ ]:
# normal      Outer normal vector of each edge
# h           Distance between two points on each side
# sample_num  Total number of samples
# sample_x    Sample point coordinates
# sample_u    Boundary condition

rot = torch.Tensor([[0,-1],[1,0]])  
a0,b0,c0,d0 = 1,1,1,1
normal = torch.zeros(sample_tri,vertex_num,2)  
h = torch.zeros(sample_tri,vertex_num,1)     
for j in range(sample_tri):
    for i in range(vertex_num):
        normal[j,i,:] = (vertex[j,(i+1)%vertex_num,:]-vertex[j,i,:])@rot
        normal[j,i,:] = normal[j,i,:]/(normal[j,i,:].norm())
        h[j,i] = (vertex[j,(i+1)%vertex_num,:]-vertex[j,i,:]).norm()/M

sample_num = vertex_num*M
sample_x = torch.zeros(sample_tri,vertex_num*M,5)
for j in range(sample_tri):
    for i in range(vertex_num):
        sample_x[j,i*M:i*M+M,0:2] = vertex[j,i,:]+(vertex[j,(i+1)%vertex_num,:]-vertex[j,i,:])*line
    sample_x[j,:,2:5] = vertex_sample[j,:]
sample_u = torch.zeros(sample_tri, sample_num,1)

In [ ]:
# Generate the integral matrix of the fundamental solution 
G1 = torch.zeros(sample_tri, sample_num,sample_num)
d = torch.zeros(sample_num,sample_num)
for p in range(sample_tri):
    for i in range(sample_num):
        for j in range(sample_num):
            r = sample_x[p,j,0:2] - sample_x[p,i,0:2]
            d[i,j] = r.norm()
    G1[p,:,:] = -1/(2*np.pi)*torch.log(d)

A1 = torch.zeros(sample_tri, sample_num,sample_num)
c = torch.zeros(sample_num,sample_num)
for p in range(sample_tri):
    for i in range(sample_num):
        if (i%M != 0) and (i%M != 1) and (i%M != M-1):
            sample_u[p,i] = d0 + b0*sample_x[p,i,0]+c0*sample_x[p,i,1]+a0*sample_x[p,i,0]*sample_x[p,i,1]
            for j in range(sample_num):
                j0 = int(j/M)
                j1 = int((j-1)/M)%vertex_num
                c[i,j] = ((h[p,j0]+h[p,j1])/2)
            A1[p,i,:]= -c[i,:]*G1[p,i,:]
            A1[p,i,i] = 0

In [5]:
net1 = Nw.ResNet2(100,5,1).to(device)
loss_func = lf.Laplace_loss()


In [ ]:
#Training 
# sample_h    The densitise on sample points

optimizer = torch.optim.Adam(net1.parameters(net1),lr=0.0005)
Epoch = 1
loss_all = np.zeros(Epoch+1)
time0 = time.time()

for epoch in range(Epoch+1):
    loss = 0
    if (epoch+1)%500==0:
        vertex_sample = (torch.rand(sample_tri,3)).to(device)
        for i in range(sample_tri):
            vertex[i,1,0] = vertex_sample[i,0]
            vertex[i,2,:] = vertex_sample[i,1:3]

        for j in range(sample_tri):
            for i in range(vertex_num):
                h[j,i] = (vertex[j,(i+1)%vertex_num,:]-vertex[j,i,:]).norm()/M

        for j in range(sample_tri):
            for i in range(vertex_num):
                sample_x[j,i*M:i*M+M,0:2] = vertex[j,i,:]+(vertex[j,(i+1)%vertex_num,:]-vertex[j,i,:])*line
            sample_x[j,:,2:5] = vertex_sample[j,:]

        for p in range(sample_tri):
            for i in range(sample_num):
                for j in range(sample_num):
                    r = sample_x[p,j,0:2] - sample_x[p,i,0:2]
                    d[i,j] = r.norm()
            G1[p,:,:] = -1/(2*np.pi)*torch.log(d)
        for p in range(sample_tri):
            for i in range(sample_num):
                if (i%M != 0) and (i%M != 1) and (i%M != M-1):
                    sample_u[p,i] = d0 + b0*sample_x[p,i,0]+c0*sample_x[p,i,1]+a0*sample_x[p,i,0]*sample_x[p,i,1]
                    for j in range(sample_num):
                        j0 = int(j/M)
                        j1 = int((j-1)/M)%vertex_num
                        c[i,j] = ((h[p,j0]+h[p,j1])/2)
                    A1[p,i,:] = -c[i,:]*G1[p,i,:]
                    A1[p,i,i] = 0
            
    for i in range(sample_tri):
        sample_h= (net1(sample_x[i,:,:])).reshape(-1,1)
    
        u0 = (A1[i,:,:]@sample_h)
        loss = loss + Green_loss_func(sample_u[i,:],u0)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_all[epoch] = loss.detach().numpy() 
    if epoch%100==0:
        print('loss, epoch, computation time:','%.4f'%loss.detach().numpy(),epoch,'%.4f'%(time.time()-time0))
        time0 = time.time()



In [ ]:
# Test relative L2 error

M0 = 80
sample_num0 = vertex_num*80
line0 = (torch.linspace(0,1-1/M0,M0)).reshape(-1,1)
a0,b0,c0 = 0.8, 0.5, 0.6
vertex0 = torch.Tensor([[0,0],[a0,0],[b0,c0],[0,0]])
sample_x0 = torch.zeros(sample_num0,5)
sample_x0[:,2] = a0
sample_x0[:,3] = b0
sample_x0[:,4] = c0
for i in range(vertex_num):
    sample_x0[i*M0:i*M0+M0,0:2] = vertex0[i,:]+(vertex0[i+1,:]-vertex0[i,:])*line0

h0 = torch.zeros(vertex_num,1)
for i in range(vertex_num):
    h0[i] = (vertex0[(i+1)%vertex_num,:]-vertex0[i,:]).norm()/M0

sample_h0 = (net1(sample_x0)).reshape(-1,1)


In [ ]:
sample = 566
with open('points.txt','r') as f:
    data = f.readlines()
u_in = torch.zeros(sample,1)
x_in = torch.zeros(sample,2)
for i in range(sample):
    x_in[i,0] = float((data[i].split())[0])
    x_in[i,1] = float((data[i].split())[1])
    u_in[i] = d0 + b0*x_in[i,0]+c0*x_in[i,1]+a0*x_in[i,0]*x_in[i,1]



In [ ]:
G1_in = torch.zeros(sample,sample_num0)
for i in range(sample):
    for j in range(sample_num0):
        j0 = int(j/M0)
        j1 = int((j-1)/M0)%vertex_num
        r = sample_x1[j,0:2]-x_in[i,:]
        d = r.norm()
        G1_in[i,j] = -1/(2*np.pi)*torch.log(d)*(h0[j0]+h0[j1])/2

In [ ]:
u_green = -(G1_in@sample_h0)
print((u_green-u_in).norm()/u_in.norm())